## Linking and brushing with bokeh

Linking and brushing is a powerful method for exploratory data analysis.

One way to create linked plots in the notebook is to use Bokeh.

In [1]:
import bokeh

In [2]:
import numpy as np

In [3]:
from astropy.table import Table

In [4]:
sdss = Table.read('data/sdss_galaxies_qsos_50k.fits')

In [5]:
sdss

ra,dec,u,g,r,i,z,class,specz,speczer
float64,float64,float32,float32,float32,float32,float32,str32,float32,float32
55.5854993277,1.0214755259,18.7981,17.1568,16.4629,16.0512,15.7572,GALAXY,0.0375854,1.21012e-05
55.5936253842,1.00002821365,19.2083,18.0934,17.6449,17.3163,17.1272,GALAXY,0.0722986,6.83051e-06
119.864533502,0.577223091692,19.5077,17.588,16.6191,16.1577,15.8404,GALAXY,0.0949418,1.31523e-05
244.284910359,-0.455476619333,18.6243,16.6891,15.6732,15.2011,14.8175,GALAXY,0.092808,1.84619e-05
244.241553069,-0.048437840441,19.3458,17.9135,17.2692,16.9746,16.6808,GALAXY,0.0803378,1.2551e-05
197.212436004,0.405354382508,18.9968,17.0615,16.0763,15.6325,15.2942,GALAXY,0.096066,1.21444e-05
242.273332856,1.08297757452,19.3646,18.0262,17.5317,17.2323,17.0368,GALAXY,0.0553339,5.40396e-06
189.393474701,-1.24580574622,19.5096,17.3705,16.261,15.8153,15.451,GALAXY,0.135351,2.36187e-05
189.999155244,-1.17914420262,18.557,18.2298,18.1817,18.3339,18.2263,QSO,0.819497,0.000170763


In [6]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, gridplot, output_notebook, output_file, show


umg = sdss['u'] - sdss['g']
gmr = sdss['g'] - sdss['r']
rmi = sdss['r'] - sdss['i']
imz = sdss['i'] - sdss['z']

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(umg=umg, gmr=gmr, rmi=rmi,imz=imz))

We will output to a static html file. 

The output_notebook() function can output to the notebook, 
but with 50,000 points it really slows down.

In [7]:
output_file('sdss_color_color.html')

In [8]:
TOOLS = "pan,wheel_zoom,reset,box_select,poly_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=400, height=400, title='SDSS g-r vs u-g', webgl=True)
left.x('umg', 'gmr', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=400, height=400, title='SDSS i-z vs r-i')
right.x('rmi', 'imz', source=source)

p = gridplot([[left, right]])

show(p)

See many examples of configuring plot tools at http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html

## Interacting with Glue

In [9]:
#import glue

In [ ]:
# Quick way to launch Glue
#from glue import qglue
#qglue()

Here we'll interact with Glue from the notebook.

In [11]:
import astropy.io.fits as fits
hdu = fits.open('data/w5.fits')

In [12]:
hdu[0].header

SIMPLE  =                    T / Written by SkyView Mon Sep 24 16:13:12 EDT 2012
BITPIX  =                  -32 / 4 byte floating point                          
NAXIS   =                    2 / Two dimensional image                          
NAXIS1  =                 1000 / Width of image                                 
NAXIS2  =                 1000 / Height of image                                
CRVAL1  =                43.46 / Reference longitude                            
CRVAL2  =                60.38 / Reference latitude                             
RADESYS = 'FK5     '           / Coordinate system                              
EQUINOX =               2000.0 / Epoch of the equinox                           
CTYPE1  = 'RA---TAN'           / Coordinates -- projection                      
CTYPE2  = 'DEC--TAN'           / Coordinates -- projection                      
CRPIX1  =                500.5 / X reference pixel                              
CRPIX2  =                500

In [13]:
from astropy.table import Table

In [14]:
catalog = Table.read('data/w5_psc.vot')

In [15]:
wisecat = Table.read('data/w5_wise.tbl', format='ipac')

In [16]:
%gui qt

In [17]:
#qglue(catalog=catalog, image=hdu, wisecat=wisecat)

In [18]:
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication

#load 2 datasets from files
image = load_data('data/w5.fits')
catalog = load_data('data/w5_psc.vot')

dc = DataCollection([image, catalog])

# link positional information
dc.add_link(LinkSame(image.id['Right Ascension'], catalog.id['RAJ2000']))
dc.add_link(LinkSame(image.id['Declination'], catalog.id['DEJ2000']))

#start Glue
app = GlueApplication(dc)
app.start()

-1

In [19]:
dc

DataCollection (2 data sets)
	  0: w5[PRIMARY]
	  1: w5_psc

In [20]:
dc[0].components

[Declination, PRIMARY, Pixel x, Pixel y, Right Ascension]

In [21]:
dc[0].id['Right Ascension']

Right Ascension

In [22]:
catalog = dc[1]

In [23]:
j_minus_h = catalog['Jmag'] - catalog['Hmag']

In [24]:
catalog['jmh'] = j_minus_h

In [28]:
hmag = catalog.id['Hmag']
jmag = catalog.id['Jmag']

In [29]:
jmhred = (jmag - hmag) > 1.5
dc.new_subset_group('j - h > 1.5', mid_mag)

In [30]:
dc.subset_groups

(<glue.core.subset_group.SubsetGroup at 0x12e567cf8>,
 <glue.core.subset_group.SubsetGroup at 0x12fb39748>)

In [31]:
dc.subset_groups[2].label

'hmag_ between_10_15'

In [32]:
catalog.subsets

(Subset: 1.3 < jmh < 2.6 (data: w5_psc),
 Subset: Western lobe (data: w5_psc),
 Subset: hmag_ between_10_15 (data: w5_psc),
 Subset: j - h > 1.5 (data: w5_psc))

In [33]:
catalog.subsets[2]['Jmag']

array([ 15.34000015,  10.89999962,  13.30000019, ...,  13.06000042,
        13.38000011,  14.18000031], dtype=float32)

In [34]:
catalog.subsets[2].to_mask()

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)